In [11]:
import sqlite3
import pandas as pd
import requests
import os
import re
#!pip install python-docx
from docx import Document  # Import the python-docx library

In [2]:
# Connect to the SQLite database
connection = sqlite3.connect("../data/eppocodes.sqlite")
# connection1 = sqlite3.connect("../data/eppocodes_all.sqlite")

In [3]:
# Define the query to retrieve all rows from the t_codes table
query = """--sql
SELECT * FROM t_codes
"""

# Use pandas to execute the query and load the data into a DataFrame
t_codes_df = pd.read_sql_query(query, connection)

# Display the data
print(t_codes_df)

        codeid eppocode dtcode status            c_date            m_date
0            1   ABSICO    GAF      A  1997-06-26 00:00  1997-06-26 00:00
1            2   ABSIRA    GAF      A  1997-06-26 00:00  1997-06-26 00:00
2            3   ABSIRE    GAF      A  1996-10-16 00:00  1996-10-16 00:00
3            4   ABSISP    GAF      A  1996-10-16 00:00  1996-10-16 00:00
4            6   ACCMMC    GAF      A  2002-02-27 00:00  2002-02-27 00:00
...        ...      ...    ...    ...               ...               ...
126089  128982   1PICEG    SIT      A  2024-11-14 18:30              None
126090  128983   PICESP    GAI      A  2024-11-14 18:30              None
126091  128984   1PSEMG    SIT      A  2024-11-14 18:32              None
126092  128985    ACAKN    PFL      A  2024-11-14 18:46              None
126093  128986    WAZDI    PFL      A  2024-11-14 18:49              None

[126094 rows x 6 columns]


In [6]:
test_url = "https://gd.eppo.int/taxon/LIBEAF/download/datasheet_doc?action=custom&list%5B%5D=identity&list%5B%5D=hosts&list%5B%5D=geog&list%5B%5D=biology&list%5B%5D=detection&list%5B%5D=movement&list%5B%5D=significance&list%5B%5D=phytosanitary"
# Define the folder to save the downloaded files
output_folder = "../data/eppo_downloads"

output_file = os.path.join(output_folder, "LIBEAF.docx")
response = requests.get(test_url)


with open(output_file, "wb") as file:
    file.write(response.content)
    
    
test_url = "https://gd.eppo.int/taxon/PSDMAC/download/datasheet_doc?action=custom&list%5B%5D=identity&list%5B%5D=hosts&list%5B%5D=geog&list%5B%5D=biology&list%5B%5D=detection&list%5B%5D=movement&list%5B%5D=significance&list%5B%5D=phytosanitary"
# Define the folder to save the downloaded files
output_folder = "../data/eppo_downloads"

output_file = os.path.join(output_folder, "PSDMAC.docx")
response = requests.get(test_url)


with open(output_file, "wb") as file:
    file.write(response.content)

In [13]:
# Define the base URL for downloading
base_url = "https://gd.eppo.int/taxon/{}/download/datasheet_doc?action=custom&list%5B%5D=identity&list%5B%5D=hosts&list%5B%5D=geog&list%5B%5D=biology&list%5B%5D=detection&list%5B%5D=movement&list%5B%5D=significance&list%5B%5D=phytosanitary"

# Define the folder to save the downloaded files
output_folder = "../data/eppo_downloads"
os.makedirs(output_folder, exist_ok=True)

# Loop through each EPPO code in the DataFrame
for eppocode in t_codes_df['eppocode']:
    # Construct the URL for each EPPO code
    download_url = base_url.format(eppocode)
    
    # Define the output file path
    output_file = os.path.join(output_folder, f"{eppocode}.docx")
    
    try:
        # Send a GET request to download the file
        response = requests.get(download_url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Write the content to a file
            with open(output_file, "wb") as file:
                file.write(response.content)
            print(f"Downloaded: {output_file}")
        else:
            print(f"Failed to download {eppocode}: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading {eppocode}: {e}")

Downloaded: data/eppo_downloads\ABSICO.docx
Downloaded: data/eppo_downloads\ABSIRA.docx
Downloaded: data/eppo_downloads\ABSIRE.docx
Downloaded: data/eppo_downloads\ABSISP.docx
Downloaded: data/eppo_downloads\ACCMMC.docx
Downloaded: data/eppo_downloads\ACETAC.docx
Downloaded: data/eppo_downloads\ACETSP.docx
Downloaded: data/eppo_downloads\ACHLSP.docx
Downloaded: data/eppo_downloads\ACHNSP.docx
Downloaded: data/eppo_downloads\ACINCA.docx
Failed to download ACLSV: HTTP 404
Failed to download ACLSVM: HTTP 404
Failed to download ACMV: HTTP 404
Failed to download ACMV.C: HTTP 404
Failed to download ACMV.T: HTTP 404
Downloaded: data/eppo_downloads\ACPHFU.docx
Downloaded: data/eppo_downloads\ACREAC.docx
Downloaded: data/eppo_downloads\ACREAP.docx
Downloaded: data/eppo_downloads\ACREBU.docx
Downloaded: data/eppo_downloads\ACREBY.docx
Downloaded: data/eppo_downloads\ACREFU.docx
Downloaded: data/eppo_downloads\ACREKI.docx
Downloaded: data/eppo_downloads\ACRERE.docx
Downloaded: data/eppo_downloads

In [14]:
# Define the folder containing the documents
input_folder = "../data/eppo_downloads"

# Initialize a list to store the extracted data
data = []

# Define the main headings to extract
main_headings = [
    "IDENTITY",
    "HOSTS",
    "GEOGRAPHICAL DISTRIBUTION",
    "BIOLOGY",
    "DETECTION AND IDENTIFICATION",
    "PATHWAYS FOR MOVEMENT",
    "PEST SIGNIFICANCE",
    "PHYTOSANITARY MEASURES"
]

# Function to parse a .docx document
def parse_document(file_path):
    # Load the .docx file
    doc = Document(file_path)
    
    # Extract the document content as plain text
    content = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    
    # Extract document name (ID)
    document_id = os.path.splitext(os.path.basename(file_path))[0]
    
    # Extract full name from the title
    title_match = re.search(r"EPPO Datasheet: (.*?)\n", content)
    full_name = title_match.group(1) if title_match else "Unknown"
    
    # Split the document into sections based on the main headings
    sections = re.split(r"\n(?=(" + "|".join(main_headings) + r"))", content)
    for i in range(1, len(sections), 2):  # Process the heading and its content
        section_title = sections[i].strip()
        section_content = sections[i + 1].strip() if i + 1 < len(sections) else ""
        
        # Append the data
        data.append({
            "Document ID": document_id,
            "Full Name": full_name,
            "Section Title": section_title,
            "Section Content": section_content
        })

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".docx"):  # Ensure you're only processing .docx files
        file_path = os.path.join(input_folder, filename)
        parse_document(file_path)

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data)

# Save to a CSV file for reference
output_csv = "../data/eppo_documents_filtered.csv"
df.to_csv(output_csv, index=False)

# Display the dataframe
print(df.head())

  Document ID                            Full Name                 Section Title                                    Section Content
0      LIBEAF  'Candidatus Liberibacter africanus'                      IDENTITY  IDENTITY\n\nNotes on taxonomy and nomenclature...
1      LIBEAF  'Candidatus Liberibacter africanus'                         HOSTS  HOSTS\nThe three species of ‘Ca. Liberibacter’...
2      LIBEAF  'Candidatus Liberibacter africanus'     GEOGRAPHICAL DISTRIBUTION  GEOGRAPHICAL DISTRIBUTION\nHuanglongbing was p...
3      LIBEAF  'Candidatus Liberibacter africanus'                       BIOLOGY  BIOLOGY\nLocation within the plant\nThe three ...
4      LIBEAF  'Candidatus Liberibacter africanus'  DETECTION AND IDENTIFICATION  DETECTION AND IDENTIFICATION\nSymptoms\nOn inf...
